# PROVES Library - Setup and Exploration

This notebook will help you:
1. Test the Neon PostgreSQL connection
2. Apply the database schema (knowledge graph tables)
3. Index your first library entry
4. Explore the knowledge graph
5. Prototype the Curator agent

---

## Prerequisites

- Neon MCP server connected (23 tools available)
- `.env` file with `NEON_DATABASE_URL` filled in
- Python virtual environment activated

## 1. Setup and Imports

In [ ]:
import sys
import os
from pathlib import Path

# Add scripts directory to path
project_root = Path.cwd().parent
scripts_dir = project_root / 'scripts'
sys.path.insert(0, str(scripts_dir))

# Load environment variables
from dotenv import load_dotenv
load_dotenv(project_root / '.env')

print(f"✅ Project root: {project_root}")
print(f"✅ Scripts directory added to path")
print(f"✅ Environment loaded")

In [ ]:
# Import our utilities
from db_connector import get_db
from graph_manager import GraphManager
from library_indexer import LibraryIndexer

# For visualization
import pandas as pd
import json

print("✅ All utilities imported successfully")

## 2. Test Database Connection

In [ ]:
# Test connection to Neon
db = get_db()

# Get PostgreSQL version
result = db.fetch_one("SELECT version() as version")
print(f"✅ Connected to PostgreSQL")
print(f"\nVersion: {result['version'][:80]}...")

# Get database name
db_info = db.fetch_one("SELECT current_database() as db_name, current_user as user_name")
print(f"\nDatabase: {db_info['db_name']}")
print(f"User: {db_info['user_name']}")

## 3. Apply Database Schema

This will create all the tables for:
- Knowledge graph (nodes and relationships)
- Library entries
- Risk patterns and scans
- Agent workflows (Curator and Builder)

**Note:** This will only run if the tables don't already exist.

In [ ]:
# Check if schema already exists
check_query = """
SELECT EXISTS (
    SELECT FROM information_schema.tables 
    WHERE table_name = 'library_entries'
) as schema_exists
"""

result = db.fetch_one(check_query)

if result['schema_exists']:
    print("⚠️  Schema already exists. Skipping schema creation.")
    print("If you need to reset, manually drop tables in Neon console.")
else:
    print("📝 Schema not found. Applying schema files...")
    print("\nThis may take 30-60 seconds...")

In [ ]:
# Apply schema if needed
if not result['schema_exists']:
    schema_dir = project_root / 'mcp-server' / 'schema'
    
    # Read and execute 00_initial_schema.sql
    print("📄 Applying 00_initial_schema.sql...")
    with open(schema_dir / '00_initial_schema.sql', 'r') as f:
        schema_sql = f.read()
    
    db.execute(schema_sql)
    print("✅ Initial schema applied")
    
    # Read and execute 01_seed_data.sql
    print("\n📄 Applying 01_seed_data.sql...")
    with open(schema_dir / '01_seed_data.sql', 'r') as f:
        seed_sql = f.read()
    
    db.execute(seed_sql)
    print("✅ Seed data applied")
    
    print("\n🎉 Database schema initialized successfully!")

## 4. Verify Schema Installation

In [ ]:
# Get database statistics
stats = db.fetch_all("SELECT * FROM database_statistics ORDER BY table_name")
stats_df = pd.DataFrame(stats)

print("📊 Database Statistics:\n")
print(stats_df.to_string(index=False))

total_rows = stats_df['row_count'].sum()
print(f"\nTotal rows across all tables: {total_rows}")

## 5. Explore Initial Seed Data

The seed data includes:
- 5 risk patterns (I2C conflict, memory leak, power budget, etc.)
- Example hardware nodes (MPU-6050 IMU, BNO055 IMU, TCA9548A multiplexer)
- Example F´ components
- Example relationships showing conflicts and dependencies

In [ ]:
# View risk patterns
risk_patterns = db.fetch_all("""
    SELECT name, pattern_type, severity, detection_method, fix_summary
    FROM risk_patterns
    ORDER BY 
        CASE severity
            WHEN 'critical' THEN 1
            WHEN 'high' THEN 2
            WHEN 'medium' THEN 3
            WHEN 'low' THEN 4
        END
""")

print("🔍 Risk Patterns:\n")
risk_df = pd.DataFrame(risk_patterns)
print(risk_df.to_string(index=False))

In [ ]:
# View knowledge graph nodes
gm = GraphManager()
stats = gm.get_statistics()

print("🌐 Knowledge Graph Statistics:\n")
print(f"Total Nodes: {stats['total_nodes']}")
print(f"Total Relationships: {stats['total_relationships']}")

if stats['nodes_by_type']:
    print("\nNodes by Type:")
    for node_type, count in stats['nodes_by_type'].items():
        print(f"  {node_type}: {count}")

if stats['relationships_by_type']:
    print("\nRelationships by Type:")
    for rel_type, count in stats['relationships_by_type'].items():
        print(f"  {rel_type}: {count}")

In [ ]:
# View hardware nodes in detail
hardware = gm.search_nodes(node_type='hardware')

print("🔧 Hardware Components:\n")
for node in hardware:
    print(f"\n{node['name']}")
    print(f"  Description: {node['description']}")
    print(f"  Properties:")
    props = node['properties']
    for key, value in props.items():
        print(f"    {key}: {value}")

## 6. Explore Relationships (ERV)

The Engineering Relationship Vocabulary (ERV) defines 6 relationship types:
- `depends_on` - Component A depends on Component B
- `conflicts_with` - Component A conflicts with Component B
- `enables` - Component A enables capability B
- `requires` - Component A requires condition/config B
- `mitigates` - Solution A mitigates risk B
- `causes` - Action A causes consequence B

In [ ]:
# View all relationships
relationships = db.fetch_all("""
    SELECT 
        sn.name as source,
        r.relationship_type,
        tn.name as target,
        r.description,
        r.is_critical
    FROM kg_relationships r
    JOIN kg_nodes sn ON r.source_node_id = sn.id
    JOIN kg_nodes tn ON r.target_node_id = tn.id
    ORDER BY r.is_critical DESC, r.relationship_type
""")

print("🔗 Knowledge Graph Relationships:\n")
for rel in relationships:
    critical = "⚠️ " if rel['is_critical'] else ""
    print(f"{critical}{rel['source']} --[{rel['relationship_type']}]--> {rel['target']}")
    print(f"  {rel['description']}")
    print()

## 7. Index Library Entry

Now let's index the existing I2C conflict example from the library.

In [ ]:
# Initialize indexer
indexer = LibraryIndexer()

print("📚 Indexing library entries...\n")
stats = indexer.index_all(verbose=True)

In [ ]:
# View indexed entries
entries = db.fetch_all("""
    SELECT 
        title,
        entry_type,
        domain,
        array_length(tags, 1) as tag_count,
        array_length(sources, 1) as source_count,
        quality_tier,
        created_at
    FROM library_entries
    ORDER BY created_at DESC
""")

print("\n📖 Library Entries:\n")
entries_df = pd.DataFrame(entries)
print(entries_df.to_string(index=False))

In [ ]:
# View entry details
entry = db.fetch_one("""
    SELECT *
    FROM library_entries
    WHERE slug = 'example-i2c-conflict'
""")

if entry:
    print("📄 Entry Details:\n")
    print(f"Title: {entry['title']}")
    print(f"Type: {entry['entry_type']}")
    print(f"Domain: {entry['domain']}")
    print(f"Tags: {', '.join(entry['tags'])}")
    print(f"Sources: {len(entry['sources'])} citations")
    print(f"\nSummary:\n{entry['summary'][:300]}...")
else:
    print("Entry not found. Check if the file exists in library/software/")

## 8. Test Graph Queries

Let's test some common graph queries that the agents will use.

In [ ]:
# Search for conflict relationships
conflicts = db.fetch_all("""
    SELECT 
        sn.name as source,
        tn.name as target,
        r.description,
        r.strength
    FROM kg_relationships r
    JOIN kg_nodes sn ON r.source_node_id = sn.id
    JOIN kg_nodes tn ON r.target_node_id = tn.id
    WHERE r.relationship_type = 'conflicts_with'
    ORDER BY r.strength DESC
""")

print("⚠️  Component Conflicts:\n")
for conflict in conflicts:
    print(f"{conflict['source']} <--> {conflict['target']}")
    print(f"  Strength: {conflict['strength']}")
    print(f"  {conflict['description']}")
    print()

In [ ]:
# Find the I2C multiplexer node
multiplexer = gm.get_node_by_name('TCA9548A I2C Multiplexer', 'hardware')

if multiplexer:
    print("🔌 TCA9548A I2C Multiplexer:\n")
    print(f"ID: {multiplexer['id']}")
    print(f"Description: {multiplexer['description']}")
    
    # Get its relationships
    rels = gm.get_node_relationships(multiplexer['id'])
    
    print(f"\nRelationships ({len(rels)}):")
    for rel in rels:
        print(f"  {rel['relationship_type']}: {rel['target_name']}")
        print(f"    {rel['description']}")

## 9. Prototype: Curator Agent

Let's prototype the Curator agent's workflow:
1. Take a raw capture (unstructured text)
2. Extract citations and metadata
3. Check for duplicates in the graph
4. Generate a normalized library entry
5. Score quality

We'll use a simulated capture for now.

In [ ]:
# Simulated raw capture from a GitHub issue
raw_capture = """
We ran into a major problem during integration testing last week. 
Two IMU sensors (MPU-6050 and BNO055) both defaulted to I2C address 0x68. 
The system would hang during initialization because of the address collision.

After some research, we found that using a TCA9548A I2C multiplexer solved the issue.
We put each IMU on a separate channel of the multiplexer. 

Testing showed no performance impact. The solution has been stable for 3 weeks now.

References:
- https://github.com/example/cubesat/issues/456
- https://www.ti.com/product/TCA9548A
- Commit: abc123def
"""

print("📝 Raw Capture:\n")
print(raw_capture)

In [ ]:
# Create a curator job (for tracking)
job_query = """
    INSERT INTO curator_jobs (raw_capture_text, source_url, status, stage)
    VALUES (%s, %s, %s, %s)
    RETURNING id
"""

job = db.fetch_one(
    job_query,
    (raw_capture, 'https://github.com/example/cubesat/issues/456', 'processing', 'citation_extraction')
)

job_id = job['id']
print(f"✅ Created curator job: {job_id}")

### Step 1: Extract Citations (Simulated)

In the full agent, this would use an LLM to extract citations, hardware mentions, etc.

In [ ]:
# Simulated extraction (in real agent, this would use Claude/GPT)
extracted_metadata = {
    'title': 'I2C Address Conflict Between MPU-6050 and BNO055',
    'problem': 'Address collision on I2C bus causing system hang',
    'solution': 'Use TCA9548A I2C multiplexer to separate devices',
    'hardware_mentioned': ['MPU-6050', 'BNO055', 'TCA9548A'],
    'citations': [
        'https://github.com/example/cubesat/issues/456',
        'https://www.ti.com/product/TCA9548A'
    ],
    'verification': 'Tested stable for 3 weeks',
    'domain': 'software',
    'type': 'failure',
    'tags': ['i2c', 'hardware', 'multiplexer', 'integration']
}

print("🔍 Extracted Metadata:\n")
print(json.dumps(extracted_metadata, indent=2))

### Step 2: Check for Duplicates

In [ ]:
# Search for similar entries
similar = db.fetch_all("""
    SELECT title, slug, entry_type, domain
    FROM library_entries
    WHERE tags && %s  -- Array overlap operator
    ORDER BY array_length(tags, 1) DESC
    LIMIT 5
""", (extracted_metadata['tags'],))

print("🔍 Similar Entries:\n")
if similar:
    for entry in similar:
        print(f"  - {entry['title']} ({entry['slug']})")
    
    print("\n⚠️  Possible duplicate detected. Would flag for human review.")
else:
    print("  No similar entries found. This appears to be a new pattern.")

### Step 3: Quality Scoring (Simulated)

In [ ]:
# Simulated quality scoring
def score_quality(metadata, raw_text):
    """
    Score quality based on:
    - Citation count (0-1 scale)
    - Verification present (0-1)
    - Completeness (problem + solution + verification)
    - Hardware specificity
    """
    scores = {}
    
    # Citation score (>= 2 citations = 1.0)
    citation_count = len(metadata.get('citations', []))
    scores['citations'] = min(citation_count / 2.0, 1.0)
    
    # Verification score
    scores['verification'] = 1.0 if metadata.get('verification') else 0.0
    
    # Completeness (has problem, solution, verification)
    has_problem = bool(metadata.get('problem'))
    has_solution = bool(metadata.get('solution'))
    has_verification = bool(metadata.get('verification'))
    scores['completeness'] = (has_problem + has_solution + has_verification) / 3.0
    
    # Hardware specificity (>= 2 components mentioned)
    hw_count = len(metadata.get('hardware_mentioned', []))
    scores['hardware_specificity'] = min(hw_count / 2.0, 1.0)
    
    # Overall score (weighted average)
    overall = (
        scores['citations'] * 0.25 +
        scores['verification'] * 0.30 +
        scores['completeness'] * 0.30 +
        scores['hardware_specificity'] * 0.15
    )
    
    return overall, scores

quality_score, component_scores = score_quality(extracted_metadata, raw_capture)

print("📊 Quality Scoring:\n")
print(f"Component Scores:")
for component, score in component_scores.items():
    print(f"  {component}: {score:.2f}")

print(f"\nOverall Score: {quality_score:.2f}")

if quality_score >= 0.8:
    tier = 'high'
elif quality_score >= 0.5:
    tier = 'medium'
else:
    tier = 'low'

print(f"Quality Tier: {tier}")

### Step 4: Update Curator Job Status

In [ ]:
# Update job with quality assessment
update_query = """
    UPDATE curator_jobs
    SET 
        status = %s,
        stage = %s,
        quality_issues = %s,
        needs_human_review = %s
    WHERE id = %s
"""

quality_issues = {
    'overall_score': quality_score,
    'tier': tier,
    'possible_duplicate': len(similar) > 0,
    'recommendations': []
}

if quality_score < 0.8:
    quality_issues['recommendations'].append('Consider adding more citations')

db.execute(
    update_query,
    ('completed', 'quality_scoring', json.dumps(quality_issues), len(similar) > 0, str(job_id))
)

print(f"✅ Curator job updated")
print(f"\nRecommendations:")
for rec in quality_issues['recommendations']:
    print(f"  - {rec}")

## 10. Summary and Next Steps

### What We've Accomplished ✅

1. Connected to Neon PostgreSQL database
2. Applied knowledge graph schema (9 tables)
3. Loaded seed data (5 risk patterns, example nodes)
4. Indexed library entry from markdown
5. Explored knowledge graph relationships
6. Prototyped Curator agent workflow

### Next Steps 🚀

1. **Build Full Curator Agent** (LangGraph workflow)
   - Use Claude API for citation extraction
   - Implement duplicate detection with embeddings
   - Add human review workflow

2. **Build Builder Agent** (F´ code generation)
   - Pattern search in knowledge graph
   - Template-based code generation
   - Test generation and validation

3. **Implement Risk Scanner**
   - AST parsing for Python/C++
   - Graph-enhanced cascade detection
   - GitHub integration for PRs

4. **Add Vector Embeddings**
   - Generate embeddings for library entries
   - Semantic search implementation
   - Similarity-based duplicate detection

5. **Build MCP Server**
   - FastAPI endpoints
   - Expose graph queries via MCP
   - Integration with Claude Code

In [ ]:
# Final statistics
final_stats = db.fetch_all("SELECT * FROM database_statistics ORDER BY table_name")

print("📊 Final Database Statistics:\n")
stats_df = pd.DataFrame(final_stats)
print(stats_df.to_string(index=False))

print("\n✅ Setup complete! Ready to build the full agentic system.")